In [1]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mediapipe as mp

# Load Model
hands = mp.solutions.hands
hand_landmark = hands.Hands(max_num_hands=1)

I0000 00:00:1725502411.161625 6264562 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
# Camera frame resolution
frame_shape = (1080, 1920, 3)

W0000 00:00:1725502411.167324 6264697 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [5]:
# # Import
# import cv2
# import numpy as np

# prevxy = None
# mask = np.zeros(frame_shape, dtype='uint8') # to premanently draw
# colour = (1, 1, 1)
# thickness = 4

# # Access camera
# cap = cv2.VideoCapture(0)
# draw = mp.solutions.drawing_utils

# # Read/Show frame's from camera
# while True:
#     _, frame = cap.read() # BGR -> RGB
#     frame = cv2.flip(frame,1)

    
#     rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     op = hand_landmark.process(rgb)
    
#     if op.multi_hand_landmarks:
#         for all_landmarks in op.multi_hand_landmarks: # 
#             draw.draw_landmarks(frame, all_landmarks, hands.HAND_CONNECTIONS)
            
#             x = int(all_landmarks.landmark[8].x*frame_shape[1])
#             y = int(all_landmarks.landmark[8].y*frame_shape[0])
            
#             if prevxy != None:
#                 # draw stuf
#                 cv2.line(mask, prevxy, (x, y), colour, thickness)
#             prevxy = (x, y)
            
#     # Merge Frame and Mask
#     frame = np.where(mask, mask, frame)
    
#     cv2.imshow('Live', frame)
#     if cv2.waitKey(1) == 27: # ESC
#         break

# cap.release()

In [11]:
import cv2
import numpy as np
import mediapipe as mp


# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
draw = mp.solutions.drawing_utils

# Initialize camera
cap = cv2.VideoCapture(0)

# Read the first frame to get frame dimensions
_, frame = cap.read()
frame_shape = frame.shape

# Initialize drawing mask and previous coordinates
prevxy = None
mask = np.zeros(frame_shape, dtype='uint8')  # To permanently draw on the mask
colour = (255,255,255)  # White color for drawing
thickness = 9

# Read/Show frames from camera
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip the frame horizontally

    # Convert frame to RGB for MediaPipe processing
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = hands.process(rgb)

    if op.multi_hand_landmarks:
        for all_landmarks in op.multi_hand_landmarks:
            # Get points for index finger landmarks 5, 6, and 7
            index_A = np.array([all_landmarks.landmark[5].x, all_landmarks.landmark[5].y, all_landmarks.landmark[5].z])  # Point A
            index_B = np.array([all_landmarks.landmark[6].x, all_landmarks.landmark[6].y, all_landmarks.landmark[6].z])  # Point B
            index_C = np.array([all_landmarks.landmark[7].x, all_landmarks.landmark[7].y, all_landmarks.landmark[7].z])  # Point C

            # Get points for middle finger landmarks 9, 10, and 11
            middle_A = np.array([all_landmarks.landmark[9].x, all_landmarks.landmark[9].y, all_landmarks.landmark[9].z])  # Point A
            middle_B = np.array([all_landmarks.landmark[10].x, all_landmarks.landmark[10].y, all_landmarks.landmark[10].z])  # Point B
            middle_C = np.array([all_landmarks.landmark[11].x, all_landmarks.landmark[11].y, all_landmarks.landmark[11].z])  # Point C

            # Calculate vectors for both fingers
            vector_index_AB = index_B - index_A
            vector_index_BC = index_C - index_B
            vector_middle_AB = middle_B - middle_A
            vector_middle_BC = middle_C - middle_B

            # Calculate dot product and magnitudes for both fingers
            dot_product_index = np.dot(vector_index_AB, vector_index_BC)
            magnitude_index_AB = np.linalg.norm(vector_index_AB)
            magnitude_index_BC = np.linalg.norm(vector_index_BC)
            cosine_angle_index = dot_product_index / (magnitude_index_AB * magnitude_index_BC)
            cosine_angle_index = np.clip(cosine_angle_index, -1.0, 1.0)
            angle_degrees_index = np.degrees(np.arccos(cosine_angle_index))

            dot_product_middle = np.dot(vector_middle_AB, vector_middle_BC)
            magnitude_middle_AB = np.linalg.norm(vector_middle_AB)
            magnitude_middle_BC = np.linalg.norm(vector_middle_BC)
            cosine_angle_middle = dot_product_middle / (magnitude_middle_AB * magnitude_middle_BC)
            cosine_angle_middle = np.clip(cosine_angle_middle, -1.0, 1.0)
            angle_degrees_middle = np.degrees(np.arccos(cosine_angle_middle))

            # Get the tips of the index and middle fingers
            index_tip_x = int(all_landmarks.landmark[8].x * frame_shape[1])
            index_tip_y = int(all_landmarks.landmark[8].y * frame_shape[0])
            middle_tip_x = int(all_landmarks.landmark[12].x * frame_shape[1])
            middle_tip_y = int(all_landmarks.landmark[12].y * frame_shape[0])

            # Check if the fingers are straight (angle close to 180 degrees)
            is_index_straight = abs(angle_degrees_index % 180) <= 16
            is_middle_straight = abs(angle_degrees_middle % 180) <= 16

            # Perform the appropriate action based on the finger positions
            if is_middle_straight and not is_index_straight:
                # Middle finger is straight, index finger is bent: erase
                cv2.circle(frame, (middle_tip_x, middle_tip_y), 30, (0, 0, 0), -1)
                cv2.circle(mask, (middle_tip_x, middle_tip_y), 30, (0, 0, 0), -1)
            elif is_index_straight and not is_middle_straight:
                # Index finger is straight, middle finger is bent: draw
                draw.draw_landmarks(frame, all_landmarks, mp_hands.HAND_CONNECTIONS)
                if prevxy is not None:
                    cv2.line(mask, prevxy, (index_tip_x, index_tip_y), colour, thickness)
                prevxy = (index_tip_x, index_tip_y)
            else:
                # Neither finger is straight or both are straight: do nothing
                prevxy = None


    # Merge Frame and Mask
    frame = cv2.add(frame, mask)  # Use cv2.add to merge the mask with the frame
    
    # Show the live frame
    cv2.imshow('Live', frame)
    if cv2.waitKey(1) == 27:  # ESC key
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1725817415.179384 6264562 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1725817415.194441 7714692 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725817415.202195 7714692 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
